## SQL에서 Dataframe으로

- sqlite3 드라이버를 사용해서 SQLite 데이터베이스를 이용할 수 있다.
- https://docs.python.org/2/library/sqlite3.html

### Sqlite3 Driver

In [7]:
import sqlite3

In [8]:
query = """
    CREATE Table IF NOT EXISTS BestSellers
    (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT,
        author TEXT,
        price REAL,
        edition INTEGER
    );
"""

con = sqlite3.connect("/Users/grace/workspace/SQLite/databases/books.db")
con.execute(query)
con.commit()

In [12]:
# 데이터 입력
# 데이터는 튜플 형태로 작성한다
best_selling_amazon_books = [('Fire and Fury', 'Michael Wolff', 14.99, 1),
                             ('12 Rules for Life: An antidote to Chaos', 'Jordan B. Peterson', 15.54, 1),
                             ('A Higher Loyalty: Truth, Lies, and Leadership', 'James Comey', 8.47, 1)]

insert_query = """
    INSERT INTO BestSellers (title, author, price, edition)
    VALUES(?, ?, ?, ?)
"""

con.executemany(insert_query, best_selling_amazon_books)
con.commit()

In [28]:
# 테이터 추출
# 1개짜리
select_one_query = "SELECT * from BestSellers where title like '%Fire and Fury%';"

cursor = con.execute(select_one_query)
r = cursor.fetchone()
r

(1, 'Fire and Fury', 'Michael Wolff', 14.99, 1)

In [29]:
# cursor에서 하나씩 꺼낸 값의 자료형을 확인해보겠습니다
type(r)

tuple

In [42]:
# 데이터 추출
# 특정 컬럼 값만 추출해보겠습니다
select_query = "SELECT title, price FROM BestSellers;"

cursor = con.execute(select_query)

rows = cursor.fetchall()
rows

[('Fire and Fury', 14.99),
 ('12 Rules for Life: An antidote to Chaos', 15.54),
 ('A Higher Loyalty: Truth, Lies, and Leadership', 8.47)]

In [43]:
# 마지막 쿼리에 활용된 컬럼의 이름을 확인 할 수 있다
cursor.description

(('title', None, None, None, None, None, None),
 ('price', None, None, None, None, None, None))

In [44]:

# 입력과 동일하게 튜플 형태로 반환된다
select_query2 = "SELECT * from BestSellers;"

cursor = con.execute(select_query2)

rows2 = cursor.fetchall()
rows2

[(1, 'Fire and Fury', 'Michael Wolff', 14.99, 1),
 (2,
  '12 Rules for Life: An antidote to Chaos',
  'Jordan B. Peterson',
  15.54,
  1),
 (3, 'A Higher Loyalty: Truth, Lies, and Leadership', 'James Comey', 8.47, 1)]

In [45]:
cursor.description

(('id', None, None, None, None, None, None),
 ('title', None, None, None, None, None, None),
 ('author', None, None, None, None, None, None),
 ('price', None, None, None, None, None, None),
 ('edition', None, None, None, None, None, None))

In [46]:
# cursor.description에서 컬럼 목록을 뽑고 싶습니다
col1, col2, col3, col4, col5, col6, col7 = zip(*cursor.description)

In [47]:
col1

('id', 'title', 'author', 'price', 'edition')

In [48]:
# 하지만 위의 방식은 너무 힘드니까..
cursor_list = list(zip(*cursor.description))

In [49]:
cursor_list

[('id', 'title', 'author', 'price', 'edition'),
 (None, None, None, None, None),
 (None, None, None, None, None),
 (None, None, None, None, None),
 (None, None, None, None, None),
 (None, None, None, None, None),
 (None, None, None, None, None)]

In [50]:
columns = cursor_list[0]
columns

('id', 'title', 'author', 'price', 'edition')

### DataFrame으로 변환

In [17]:
import pandas as pd

In [55]:
df_best_sellers = pd.DataFrame(rows2, columns=columns)

In [56]:
df_best_sellers

,id,title,author,price,edition
0,1,Fire and Fury,Michael Wolff,14.99,1
1,2,12 Rules for Life: An antidote to Chaos,Jordan B. Peterson,15.54,1
2,3,"A Higher Loyalty: Truth, Lies, and Leadership",James Comey,8.47,1


In [57]:
# Database의 식별자를 index로 활용하고 싶습니다
df_best_sellers.set_index('id', inplace=True)

In [58]:
df_best_sellers

,title,author,price,edition
id,,,,
1,Fire and Fury,Michael Wolff,14.99,1
2,12 Rules for Life: An antidote to Chaos,Jordan B. Peterson,15.54,1
3,"A Higher Loyalty: Truth, Lies, and Leadership",James Comey,8.47,1


### read_sql_query()

- 위의 단계를 간편하게 할 수 있도록 pandas에서는 read_sql_query라는 함수로 쿼리문과 데이터베이스 커넥션을 인자로 넘기는 것이 가능하다
- pandas.io.sql

In [59]:
import pandas.io.sql as sql

In [60]:
sql.read_sql_query(select_query, con)

,title,price
0,Fire and Fury,14.99
1,12 Rules for Life: An antidote to Chaos,15.54
2,"A Higher Loyalty: Truth, Lies, and Leadership",8.47


In [61]:
sql.read_sql_query('SELECT * FROM BestSellers WHERE price > 10;', con)

,id,title,author,price,edition
0,1,Fire and Fury,Michael Wolff,14.99,1
1,2,12 Rules for Life: An antidote to Chaos,Jordan B. Peterson,15.54,1


### execute()

In [62]:
sql.execute('update BestSellers set edition=2', con)

In [64]:
# 데이터가 제대로 없데이트 되었는지 확인
update_data = sql.read_sql_query(select_query2, con)
update_data

,id,title,author,price,edition
0,1,Fire and Fury,Michael Wolff,14.99,2
1,2,12 Rules for Life: An antidote to Chaos,Jordan B. Peterson,15.54,2
2,3,"A Higher Loyalty: Truth, Lies, and Leadership",James Comey,8.47,2


In [66]:
update_data.set_index('id', inplace=True)

In [67]:
update_data

,title,author,price,edition
id,,,,
1,Fire and Fury,Michael Wolff,14.99,2
2,12 Rules for Life: An antidote to Chaos,Jordan B. Peterson,15.54,2
3,"A Higher Loyalty: Truth, Lies, and Leadership",James Comey,8.47,2


In [74]:
# 새로운 데이터 추가
update_data.loc[4] = ['new book', 'new author', 20.55, 1]
update_data

,title,author,price,edition
id,,,,
1,Fire and Fury,Michael Wolff,14.99,2
2,12 Rules for Life: An antidote to Chaos,Jordan B. Peterson,15.54,2
3,"A Higher Loyalty: Truth, Lies, and Leadership",James Comey,8.47,2
4,new book,new author,20.55,1


### 데이터 테이블에 저장하기

In [75]:
# if_exists는 기존 테이블이 존재할 때 어떻게 처리 할지 선택할 수 있습니다
# fail = 해당 테이블이 존재한다면 아무것도 하지 않는다
# replace = 기존 테이블을 삭제하고 새로운 데이터를 삽입
# append = 데이터만을 추가
update_data.to_sql('BestSellers', con, if_exists='replace', index=True)

In [76]:
sql.read_sql_query(select_query2, con)

,id,title,author,price,edition
0,1,Fire and Fury,Michael Wolff,14.99,2
1,2,12 Rules for Life: An antidote to Chaos,Jordan B. Peterson,15.54,2
2,3,"A Higher Loyalty: Truth, Lies, and Leadership",James Comey,8.47,2
3,4,new book,new author,20.55,1


### 대형 DataFrame을 만들기

- SQLite의 경우 그럴 경우가 매우 희박하지만, 다른 RDMBS를 활용하는 경우 데이터의 양이 많아지면 메모리 부족으로 DataFrame 생성이 안될 수 있다.

In [79]:
for chunk in sql.read_sql_query(select_query2, con, chunksize=1):
    print(chunk)
    print('****')

   id          title         author  price  edition
0   1  Fire and Fury  Michael Wolff  14.99        2
****
   id                                    title              author  price  \
0   2  12 Rules for Life: An antidote to Chaos  Jordan B. Peterson  15.54   

   edition  
0        2  
****
   id                                          title       author  price  \
0   3  A Higher Loyalty: Truth, Lies, and Leadership  James Comey   8.47   

   edition  
0        2  
****
   id     title      author  price  edition
0   4  new book  new author  20.55        1
****
